In [26]:
## use this file to return rushing statistis for carries and / or receptions per game

In [27]:
import nfl_data_py as nfl
import pandas as pd

# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set the display width to a large number

import numpy as np
from IPython.display import HTML

In [28]:
# Base columns for all positions
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team', 'opponent_team',
    'fantasy_points', 'fantasy_points_ppr'
]
print(base_columns)

['season', 'season_type', 'week', 'player_id', 'player_name', 'position', 'position_group', 'recent_team', 'opponent_team', 'fantasy_points', 'fantasy_points_ppr']


In [29]:
# RB-specific columns (rushing-related)
rb_specific_columns = [
    'carries', 'attempts','targets','receptions','rushing_yards', 'rushing_tds', 'rushing_fumbles', 
    'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 
    'rushing_2pt_conversions'
]

# Combine base columns with RB-specific columns
rb_columns = base_columns + rb_specific_columns
print(rb_columns)

['season', 'season_type', 'week', 'player_id', 'player_name', 'position', 'position_group', 'recent_team', 'opponent_team', 'fantasy_points', 'fantasy_points_ppr', 'carries', 'attempts', 'targets', 'receptions', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions']


In [30]:
# Import all relevant data for all positions
all_data = nfl.import_weekly_data(
    years=[2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017], 
    columns=base_columns + rb_specific_columns 
)

Downcasting floats.


In [31]:
# Filter to show only RB positions
rb_col_only = all_data[all_data['position'] == 'RB']

# Select only the relevant columns for RBs
rb_col_only = rb_col_only[rb_columns]

# Sort by week for easier viewing
rb_col_only = rb_col_only.sort_values(by='week')

In [32]:
# Function to export RB data grouped by season and week to a CSV file
def rb_weekly_data_export_to_csv():
    
    # Sort the data by season in descending order and week in ascending order
    rb_col_only_sorted = rb_col_only.sort_values(by=['season', 'week'], ascending=[False, True])
    
    # Export the sorted DataFrame to a CSV file
    # rb_col_only_sorted.to_csv('rb_weekly_data.csv', index=False)
    
    # Display a message confirming export
    print("Data has been exported to 'rb_weekly_data.csv'.")
    
    # Optionally display the DataFrame for verification
    display(rb_col_only_sorted)

# Call the function to execute the export
rb_weekly_data_export_to_csv()

Data has been exported to 'rb_weekly_data.csv'.


,season,season_type,week,player_id,player_name,position,position_group,recent_team,opponent_team,fantasy_points,fantasy_points_ppr,carries,attempts,targets,receptions,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions
12,2024,REG,1,00-0030578,C.Patterson,RB,RB,PIT,ATL,1.3,1.3,4,0,0,0,13.0,0,0.0,0.0,0.0,-1.204841,0
309,2024,REG,1,00-0039361,B.Irving,RB,RB,TB,WAS,7.6,9.6,9,0,3,2,62.0,0,0.0,0.0,2.0,0.843798,0
249,2024,REG,1,00-0038542,B.Robinson,RB,RB,ATL,PIT,11.1,16.1,18,0,5,5,68.0,0,0.0,0.0,5.0,-0.767227,0
252,2024,REG,1,00-0038555,T.Bigsby,RB,RB,JAX,MIA,7.3,7.3,12,0,0,0,73.0,0,0.0,0.0,2.0,-0.612941,0
258,2024,REG,1,00-0038597,C.Brown,RB,RB,CIN,NE,2.3,5.3,3,0,3,3,11.0,0,0.0,0.0,0.0,-0.348769,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2017,POST,21,00-0028087,D.Lewis,RB,RB,NE,PHI,3.9,3.9,9,0,0,0,39.0,0,0.0,0.0,1.0,0.038078,0
2619,2017,POST,21,00-0031062,J.White,RB,RB,NE,PHI,12.6,14.6,7,0,6,2,45.0,1,0.0,0.0,2.0,2.528161,0
2110,2017,POST,21,00-0030288,R.Burkhead,RB,RB,NE,PHI,6.4,7.4,3,0,1,1,18.0,0,0.0,0.0,1.0,0.735769,0
3241,2017,POST,21,00-0031590,J.Ajayi,RB,RB,PHI,NE,5.7,5.7,9,0,0,0,57.0,0,0.0,0.0,2.0,1.811190,0


In [33]:
# function returns RBs with at least 12 carries per game and ranked in descending order
def rbs_12plus_carries_per_game():
    # Sort the data by season in descending order and week in ascending order
    rb_col_only_sorted = rb_col_only.sort_values(by=['season', 'week'], ascending=[False, True])
    
    # Group by year and player, summing the carries and counting the number of games
    grouped_data = rb_col_only_sorted.groupby(
        ['season', 'season_type', 'player_id', 'player_name', 'position', 'recent_team'], as_index=False
    ).agg({
        'carries': 'sum',
        'week': 'count',  # Week serves as a proxy for number of games played
        'targets': 'sum',
        'receptions': 'sum',
        'rushing_yards': 'sum',
        'rushing_tds': 'sum',
        'rushing_fumbles': 'sum'
    })
    
    # Calculate carries per game
    grouped_data['carries_per_game'] = grouped_data['carries'] / grouped_data['week']
    
    # Filter only players with at least 12 carries per game
    filtered_data = grouped_data[grouped_data['carries_per_game'] >= 12]
    
    # Sort by season in descending order first, then by carries per game in descending order
    filtered_data = filtered_data.sort_values(by=['season', 'carries_per_game'], ascending=[False, False])
    
    # Format carries_per_game to one decimal point
    filtered_data['carries_per_game'] = filtered_data['carries_per_game'].round(1)
    
    # Select only the required columns (without 'week')
    final_columns = [
        'season', 'season_type', 'player_id', 'player_name', 'position', 'recent_team',
        'carries', 'carries_per_game', 'targets', 'receptions', 'rushing_yards', 'rushing_tds'
    ]
    filtered_data = filtered_data[final_columns]
    
    # Export the filtered DataFrame to a CSV file
    filtered_data.to_csv('rbs_12plus_carries_per_game.csv', index=False)
    
    # Return the filtered and formatted data
    return filtered_data

# Call the function to generate the CSV file and return the result
rbs_12plus_carries_per_game()


,season,season_type,player_id,player_name,position,recent_team,carries,carries_per_game,targets,receptions,rushing_yards,rushing_tds
1286,2024,REG,00-0033897,J.Mixon,RB,HOU,30,30.0,3,3,159.0,1
1325,2024,REG,00-0037525,J.Mason,RB,SF,28,28.0,1,1,147.0,1
1311,2024,REG,00-0036875,R.Stevenson,RB,NE,25,25.0,3,3,120.0,1
1292,2024,REG,00-0034844,S.Barkley,RB,PHI,24,24.0,2,2,109.0,2
1312,2024,REG,00-0036893,N.Harris,RB,PIT,20,20.0,2,1,70.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59,2017,REG,00-0029613,D.Martin,RB,TB,138,12.5,18,9,406.0,3
10,2017,POST,00-0030513,L.Murray,RB,MIN,25,12.5,4,4,68.0,1
45,2017,REG,00-0028009,D.Murray,RB,TEN,184,12.3,47,39,659.0,6
153,2017,REG,00-0033594,C.Carson,RB,SEA,49,12.2,8,7,208.0,0


In [34]:
# function returns RBs with at least 12 carries OR receptions per game and ranked in descending order
def rbs_12plus_carries_or_receptions_per_game():
    # Sort the data by season in descending order and week in ascending order
    rb_col_only_sorted = rb_col_only.sort_values(by=['season', 'week'], ascending=[False, True])
    
    # Group by year and player, summing the carries and receptions and counting the number of games
    grouped_data = rb_col_only_sorted.groupby(
        ['season', 'season_type', 'player_id', 'player_name', 'position', 'recent_team'], as_index=False
    ).agg({
        'carries': 'sum',
        'receptions': 'sum',
        'week': 'count',  # Week serves as a proxy for number of games played
        'targets': 'sum',
        'rushing_yards': 'sum',
        'rushing_tds': 'sum',
        'rushing_fumbles': 'sum'
    })
    
    # Calculate combined carries or receptions per game
    grouped_data['carries_or_receptions_per_game'] = (grouped_data['carries'] + grouped_data['receptions']) / grouped_data['week']
    
    # Filter only players with at least 12 combined carries or receptions per game
    filtered_data = grouped_data[grouped_data['carries_or_receptions_per_game'] >= 12]
    
    # Sort by season in descending order first, then by combined carries/receptions per game in descending order
    filtered_data = filtered_data.sort_values(by=['season', 'carries_or_receptions_per_game'], ascending=[False, False])
    
    # Format carries_or_receptions_per_game to one decimal point
    filtered_data['carries_or_receptions_per_game'] = filtered_data['carries_or_receptions_per_game'].round(1)
    
    # Select only the required columns (without 'week')
    final_columns = [
        'season', 'season_type', 'player_id', 'player_name', 'position', 'recent_team',
        'carries', 'receptions', 'carries_or_receptions_per_game', 'targets', 'rushing_yards', 'rushing_tds'
    ]
    filtered_data = filtered_data[final_columns]
    
    # Export the filtered DataFrame to a CSV file
    filtered_data.to_csv('rbs_12plus_carries_or_receptions_per_game.csv', index=False)
    
    # Return the filtered and formatted data
    return filtered_data

rbs_12plus_carries_or_receptions_per_game()


,season,season_type,player_id,player_name,position,recent_team,carries,receptions,carries_or_receptions_per_game,targets,rushing_yards,rushing_tds
1286,2024,REG,00-0033897,J.Mixon,RB,HOU,30,3,33.0,3,159.0,1
1325,2024,REG,00-0037525,J.Mason,RB,SF,28,1,29.0,1,147.0,1
1311,2024,REG,00-0036875,R.Stevenson,RB,NE,25,3,28.0,3,120.0,1
1292,2024,REG,00-0034844,S.Barkley,RB,PHI,24,2,26.0,2,109.0,2
1334,2024,REG,00-0038542,B.Robinson,RB,ATL,18,5,23.0,5,68.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
36,2017,REG,00-0026164,C.Johnson,RB,ARI,45,5,12.5,6,114.0,0
96,2017,REG,00-0031577,J.Allen,RB,BAL,153,46,12.4,60,591.0,4
141,2017,REG,00-0033280,C.McCaffrey,RB,CAR,117,80,12.3,113,435.0,2
112,2017,REG,00-0032058,T.Coleman,RB,ATL,156,27,12.2,39,628.0,5


In [35]:
# function returns all RBs carries and receptions per game
def rbs_all_carries_and_receptions_per_game():
    # Sort the data by season in descending order and week in ascending order
    rb_col_only_sorted = rb_col_only.sort_values(by=['season', 'week'], ascending=[False, True])
    
    # Group by year and player, summing the carries and receptions and counting the number of games
    grouped_data = rb_col_only_sorted.groupby(
        ['season', 'season_type', 'player_id', 'player_name', 'position', 'recent_team'], as_index=False
    ).agg({
        'carries': 'sum',
        'receptions': 'sum',
        'week': 'count',  # Week serves as a proxy for number of games played
        'targets': 'sum',
        'rushing_yards': 'sum',
        'rushing_tds': 'sum',
        'rushing_fumbles': 'sum'
    })
    
    # Calculate carries per game and receptions per game
    grouped_data['carries_per_game'] = grouped_data['carries'] / grouped_data['week']
    grouped_data['receptions_per_game'] = grouped_data['receptions'] / grouped_data['week']
    
    # Sort by season in descending order first, then by carries per game in descending order
    grouped_data = grouped_data.sort_values(by=['season', 'carries_per_game'], ascending=[False, False])
    
    # Format carries_per_game and receptions_per_game to one decimal point
    grouped_data['carries_per_game'] = grouped_data['carries_per_game'].round(1)
    grouped_data['receptions_per_game'] = grouped_data['receptions_per_game'].round(1)
    
    # Select only the required columns (without 'week')
    final_columns = [
        'season', 'season_type', 'player_id', 'player_name', 'position', 'recent_team',
        'carries', 'carries_per_game', 'receptions', 'receptions_per_game', 'targets', 'rushing_yards', 'rushing_tds'
    ]
    grouped_data = grouped_data[final_columns]
    
    # Export the grouped DataFrame to a CSV file
    grouped_data.to_csv('rbs_all_carries_and_receptions_per_game.csv', index=False)
    
    # Return the grouped and formatted data
    return grouped_data

# Call the function to generate the CSV file and return the result
rbs_all_carries_and_receptions_per_game()


,season,season_type,player_id,player_name,position,recent_team,carries,carries_per_game,receptions,receptions_per_game,targets,rushing_yards,rushing_tds
1286,2024,REG,00-0033897,J.Mixon,RB,HOU,30,30.0,3,3.0,3,159.0,1
1325,2024,REG,00-0037525,J.Mason,RB,SF,28,28.0,1,1.0,1,147.0,1
1311,2024,REG,00-0036875,R.Stevenson,RB,NE,25,25.0,3,3.0,3,120.0,1
1292,2024,REG,00-0034844,S.Barkley,RB,PHI,24,24.0,2,2.0,2,109.0,2
1312,2024,REG,00-0036893,N.Harris,RB,PIT,20,20.0,1,1.0,2,70.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2017,POST,00-0030668,C.West,RB,KC,0,0.0,2,2.0,2,0.0,0
16,2017,POST,00-0031682,T.Ward,RB,ATL,0,0.0,0,0.0,1,0.0,0
47,2017,REG,00-0028063,T.Jones,RB,BUF,0,0.0,1,1.0,2,0.0,0
86,2017,REG,00-0031085,J.Prosch,RB,HOU,0,0.0,4,1.0,4,0.0,0
